# Example for analysis of NanoAOD samples

In this example we don't need any pre-processing of NanoAOD samples and can still use several tools of the tW_scattering repository.

- Get the proper normalization for samples
- Categorize different samples into process categories
- Use coffea processors for the map-reduce step
- Make "nice" histograms


In [ ]:
%load_ext autoreload
%autoreload 2

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor, hist

from processor.charge_flip_check import charge_flip_check
from Tools.config_helpers import loadConfig
from klepto.archives import dir_archive

In [ ]:
from processor.default_accumulators import desired_output, add_processes_to_output

from Tools.helpers import get_samples
from Tools.config_helpers import redirector_ucsd, redirector_fnal
from Tools.nano_mapping import make_fileset, nano_mapping

from processor.meta_processor import get_sample_meta

overwrite = False
local = True

# load the config and the cache
cfg = loadConfig()

cacheName = 'charge_flip_check'
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)

year = 2018

# get a python dictionary of all NanoAOD samples
# The samples definitions can be found in data/samples.yaml
samples = get_samples(year)

# make a fileset, taking the definitions in Tools/nano_mapping.py
fileset = make_fileset(['DY', 'TTZ', 'top',], year, redirector=redirector_ucsd, small=False)
#add back in:  

# in order for cutflows to work we need to add every process to the output accumulator
add_processes_to_output(fileset, desired_output)

histograms = sorted(list(desired_output.keys()))

meta = get_sample_meta(fileset, samples)

if local:

    exe_args = {
        'workers': 16,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.futures_executor

else:
    from Tools.helpers import get_scheduler_address
    from dask.distributed import Client, progress

    scheduler_address = get_scheduler_address()
    c = Client(scheduler_address)

    exe_args = {
        'client': c,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.dask_executor

if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    print ("I'm running now")

    output = processor.run_uproot_job(
        fileset,
        "Events",
        charge_flip_check(year=year, variations=[], accumulator=desired_output),
        exe,
        exe_args,
        chunksize=250000,
    )

    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

In [ ]:
output['totalEvents']['all']/1e6

Full fileset is 180M events, and that's basically just DY and ttbar.

In [ ]:
# import the plotting libararies: matplotlib and mplhep

import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import numpy as np


# load the functions to make a nice plot from the output histograms
# and the scale_and_merge function that scales the individual histograms
# to match the physical cross section

from plots.helpers import makePlot, scale_and_merge

# define a few axes that we can use to rebin our output histograms

N_bins         = hist.Bin('multiplicity', r'$N$', 10, -0.5, 9.5)
N_bins_red     = hist.Bin('multiplicity', r'$N$', 5, -0.5, 4.5)
pt_bins        = hist.Bin('pt', r'$p_{T}\ (GeV)$', np.array([15, 40, 60, 80, 100, 200, 300]))
eta_bins       = hist.Bin('eta', r'$\eta $', np.array([0, 0.8, 1.479, 2.5]))
phi_bins       = hist.Bin('phi', r'$\phi $', 16, -3.2, 3.2)


# define nicer labels and colors

nano_mappings = nano_mapping(year)


my_labels = {
    nano_mappings['TTW'][0]: 'all',
    nano_mappings['TTZ'][0]: 'ttZ',
    nano_mappings['DY'][0]: 'DY',
    nano_mappings['top'][0]: 't/tt+jets',
}

my_colors = {
    nano_mappings['TTW'][0]: '#8AC926',
    nano_mappings['TTZ'][0]: '#FFCA3A',
    nano_mappings['DY'][0]: '#6A4C93',
    nano_mappings['top'][0]: '#1982C4',
}


# 1D Histograms

In [ ]:
from yahist import Hist1D, Hist2D

In [ ]:
tmp1 = output['N_ele'].copy()
tmp1 = tmp1.rebin('multiplicity', N_bins_red)


tmp2 = output['N_ele2'].copy()
tmp2 = tmp2.rebin('multiplicity', N_bins_red)

h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset').values()[()].T,
    (tmp1.axis('multiplicity').edges()),
)

h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset').values()[()].T,
    (tmp2.axis('multiplicity').edges()),
)

fig, (ax1,ax2) = plt.subplots(2, sharex=True, figsize=(10,10), gridspec_kw=dict(height_ratios=[3, 1]))
h2.plot(ax=ax1, alpha=1, color="C0")
h1.plot(ax=ax1, alpha=1, color="C3")

ax1.set_yscale("log")
ax1.set_xlabel(r'$N_{electron}\ $')
ax1.set_ylabel(r'Events')

fig.legend(["flipped", "weighted"])

# Gaussian errors
#    (num/den).plot(ax=ax2,show_errors=True,label="ratio")
# Asymmetric Clopper-Pearson errors
h1.divide(h2, binomial=True).plot(ax=ax2, errors=True, label="ratio")

In [ ]:
tmp1 = output['electron_flips'].copy()
tmp1 = tmp1.rebin('multiplicity', N_bins_red)


tmp2 = output['electron_flips2'].copy()
tmp2 = tmp2.rebin('multiplicity', N_bins_red)

h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset').values()[()].T,
    (tmp1.axis('multiplicity').edges()),
)

h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset').values()[()].T,
    (tmp2.axis('multiplicity').edges()),
)


fig, (ax1,ax2) = plt.subplots(2, sharex=True, figsize=(10,10), gridspec_kw=dict(height_ratios=[3, 1]))
h2.plot(ax=ax1, alpha=0.8, color="C0")
h1.plot(ax=ax1, alpha=0.8, color="C3")

ax1.set_xlabel(r'$N_{electron}\ $')
ax1.set_ylabel(r'Events')

fig.legend(["flipped", "weighted"])

h1.divide(h2, binomial=True).plot(ax=ax2, errors=True, label="ratio")

In [ ]:
tmp1 = output['electron'].copy()
tmp1 = tmp1.rebin('pt', pt_bins)
tmp1 = tmp1.rebin('eta', eta_bins)


tmp2 = output['electron2'].copy()
tmp2 = tmp2.rebin('pt', pt_bins)
tmp2 = tmp2.rebin('eta', eta_bins)

h1 = Hist2D.from_bincounts(
    tmp1.sum('dataset').values()[()].T,
    (tmp1.axis('pt').edges(), tmp1.axis('eta').edges()),
)

h2 = Hist2D.from_bincounts(
    tmp2.sum('dataset').values()[()].T,
    (tmp2.axis('pt').edges(), tmp2.axis('eta').edges()),
)

fig, (ax1,ax2) = plt.subplots(2, sharex=True, figsize=(10,10), gridspec_kw=dict(height_ratios=[3, 1]))
h2.projection('x').plot(ax=ax1, alpha=0.8, color="C0")
h1.projection('x').plot(ax=ax1, alpha=0.8, color="C3")
h2.projection('x').plot(ax=ax1, alpha=0.8, color="C0", show_errors=True)
h1.projection('x').plot(ax=ax1, alpha=0.8, color="C3", show_errors=True)

fig.legend(["weighted", "flipped"])

ax1.set_xlabel(r'$p_{T}\ (GeV) $')
ax1.set_ylabel(r'Events')

h1.projection('x').divide(h2.projection('x'), binomial=True).plot(ax=ax2, errors=True, label="ratio")
ax2.set_ylim([0.90,1.10])

In [ ]:
tmp1 = output['electron'].copy()
tmp1 = tmp1.rebin('pt', pt_bins)
tmp1 = tmp1.rebin('eta', eta_bins)


tmp2 = output['electron2'].copy()
tmp2 = tmp2.rebin('pt', pt_bins)
tmp2 = tmp2.rebin('eta', eta_bins)

h1 = Hist2D.from_bincounts(
    tmp1.sum('dataset').values()[()].T,
    (tmp1.axis('pt').edges(), tmp1.axis('eta').edges()),
)

h2 = Hist2D.from_bincounts(
    tmp2.sum('dataset').values()[()].T,
    (tmp2.axis('pt').edges(), tmp2.axis('eta').edges()),
)

fig, (ax1,ax2) = plt.subplots(2, sharex=True, figsize=(10,10), gridspec_kw=dict(height_ratios=[3, 1]))
h2.projection('y').plot(ax=ax1, alpha=0.8, color="C0")
h1.projection('y').plot(ax=ax1, alpha=0.8, color="C3", )
h2.projection('y').plot(ax=ax1, alpha=0.8, color="C0", show_errors=True)
h1.projection('y').plot(ax=ax1, alpha=0.8, color="C3", show_errors=True)

fig.legend(["weighted", "flipped"])

ax1.set_xlabel(r'$|\eta|$')
ax1.set_ylabel(r'Events')

h1.projection('y').divide(h2.projection('y'), binomial=True).plot(ax=ax2, errors=True, label="ratio")
ax2.set_ylim([0.9,1.1])

In [ ]:
h2.errors

In [ ]:
# take the N_ele histogram out of the output, apply the x-secs from samples to the samples in fileset
# then merge the histograms into the categories defined in nano_mapping


my_hist = scale_and_merge(output['N_ele'], meta, fileset, nano_mappings)


# Now make a nice plot of the electron multiplicity.
# You can have a look at all the "magic" (and hard coded monstrosities) that happens in makePlot
# in plots/helpers.py

makePlot(my_hist, None, 'multiplicity',
         bins=N_bins_red, log=True, normalize=False, axis_label=r'$N_{electron}\ $',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/N_ele_comp'
        )


In [ ]:
# take the N_ele histogram out of the output, apply the x-secs from samples to the samples in fileset
# then merge the histograms into the categories defined in nano_mapping


my_hist = scale_and_merge(output['N_ele2'], meta, fileset, nano_mappings)

# Now make a nice plot of the electron multiplicity.
# You can have a look at all the "magic" (and hard coded monstrosities) that happens in makePlot
# in plots/helpers.py

makePlot(my_hist, None, 'multiplicity',
         bins=N_bins_red, log=True, normalize=False, axis_label=r'$N_{electron}\ (ratio)$',
         new_colors=my_colors, new_labels=my_labels,
         omit=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/N_ele_ratio'
        )


In [ ]:
my_hist = scale_and_merge(output['electron_flips'], meta, fileset, nano_mappings)

makePlot(my_hist, None, 'multiplicity',
         bins=N_bins_red, log=True, normalize=False, axis_label=r'$N_{flipped\ electron}\ (SS)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/N_flips_truth'
        )

In [ ]:
my_hist = scale_and_merge(output['electron_flips2'], meta, fileset, nano_mappings)

makePlot(my_hist, None, 'multiplicity',
         bins=N_bins_red, log=True, normalize=False, axis_label=r'$N_{flipped\ electron}\ (ratio)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['top'][0], nano_mappings['TTZ'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/N_flips_ratio'
        )

In [ ]:
my_hist = scale_and_merge(output['electron'], meta, fileset, nano_mappings)

makePlot(my_hist, None, 'pt',
         bins=pt_bins, log=True, normalize=False, axis_label=r'$p_{T}\ (leading\ electron)\ (flipped)\ (GeV)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/pt_truth'
        )

In [ ]:
my_hist = scale_and_merge(output['electron2'], meta, fileset, nano_mappings)

makePlot(my_hist, None, 'pt',
         bins=pt_bins, log=True, normalize=False, axis_label=r'$p_{T}\ (leading\ electron)\ (weighted)\ (GeV)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/pt_ratio'
        )

In [ ]:
my_hist = scale_and_merge(output['electron'], meta, fileset, nano_mappings)

makePlot(my_hist, None, 'eta',
         bins=eta_bins, log=True, normalize=False, axis_label=r'$\eta\ (leading\ electron)\ (flipped)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/eta_truth'
        )

In [ ]:
my_hist = scale_and_merge(output['electron2'], meta, fileset, nano_mappings)

makePlot(my_hist, None, 'eta',
         bins=eta_bins, log=True, normalize=False, axis_label=r'$\eta\ (leading\ electron)\ (weighted)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/eta_ratio'
        )

In [ ]:
from yahist import Hist1D, Hist2D
import numpy as np

In [ ]:
tmp1 = output['electron'].copy()
tmp1 = tmp1.rebin('eta', eta_bins)
tmp1 = tmp1.rebin('pt', pt_bins)


tmp2 = output['electron2'].copy()
tmp2 = tmp2.rebin('eta', eta_bins)
tmp2 = tmp2.rebin('pt', pt_bins)

h1 = Hist2D.from_bincounts(
    tmp1.sum('dataset').values()[()].T,
    (tmp1.axis('pt').edges(), tmp1.axis('eta').edges()),
)


h2 = Hist2D.from_bincounts(
    tmp2.sum('dataset').values()[()].T,
    (tmp2.axis('pt').edges(), tmp2.axis('eta').edges()),
)

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h1.plot(show_counts=True, equidistant='xy')
ax.set_xlabel(r'$p_{T}\ (GeV)$')
ax.set_ylabel(r'$\eta$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h2.plot(show_counts=True, equidistant='xy')
ax.set_xlabel(r'$p_{T}\ (GeV)$')
ax.set_ylabel(r'$\eta$')